In [1]:
push!(LOAD_PATH,joinpath(@__DIR__, "../src/"))
using LinearAlgebra
using StableApproxEPW

## Target of the approximation problem

We consider the Helmholtz solution in the unit disk, with wavenumber

In [2]:
k = 5;

We need to define the maximum Fourier mode number `P` in the approximation
target:

In [3]:
P = 25;

Next we construct the vector of coefficients in the basis `b_p`
for `p` in `[-P,P]`:

In [4]:
U = zeros(ComplexF64, 2P+1);
U[P+1]     = 0.5;  # This is the constant mode (`p=0`)
U[P+1 + P] = 1im;  # This is mode `p = P`

The target of the approximation problem can then be constructed as:

In [5]:
u = solution_surrogate(U; k=k);

``u`` can be evaluated at any `(r,θ)` point.
Alternatively, the target ``u`` could have been a single mode.
For instance, to get the circular wave with mode number `p=15`, simply set
``u = bp(15; k=k)``.
Of course, any other function (defined in polar coordinates) can be defined
by the user as target of the approximation problem.

## Reconstruction method

The approximation will be reconstructed by sampling the target on the
boundary of the unit disk.
To do so, we need to know now the dimension `N` of the approximation sets
that we are going to use:

In [6]:
N = 100;

We can determine the number of sampling nodes necessary for a successful
reconstruction, based on `N`, `P` (to avoid aliasing) and the oversampling
ratio `η`:

In [7]:
S = number_of_boundary_sampling_nodes(N; η=2, P=P);

The (equispaced) boundary nodes are then constructed as:

In [8]:
X = boundary_sampling_nodes(S);

The right-hand-side of the linear system can then be readily constructed:

In [9]:
b = samples_from_nodes(u, X);

## Approximation with **propagative** plane waves

We construct the approximation set of PPW:

In [10]:
Φppw = approximation_set(N; k=k);

Matrix and its factorization

In [11]:
Appw = samples_from_nodes(Φppw, X);
iAppw = RegularizedSVDPseudoInverse(Appw; ϵ=1e-14);

The coefficients of the approximation are computed:

In [12]:
ξppw = solve_via_regularizedSVD(iAppw, b);
ũppw = (r,θ) -> sum([ξi * ϕi(r,θ) for (ξi, ϕi) in zip(ξppw, Φppw)]);

Absolute error function

In [13]:
eppw = (r,θ) -> ũppw(r,θ) - u(r,θ); eppw(1,π/2)

0.3951669887709478 + 6.639957427988217e-5im

Let's compute the relative residual:

In [14]:
resppw = norm(Appw * ξppw - b) / norm(b)

0.9698196014367063

We did not obtained any accuracy whatsoever!
The reason is that the coefficients are too large:

In [15]:
nrmppw = norm(ξppw) / norm(U)

6.147290882069102e10

## Approximation with **evanescent** plane waves

We construct the approximation set of EPW:

In [16]:
Φepw = approximation_set(N, P, sobol_sampling; k=k);

Matrix and its factorization

In [17]:
Aepw = samples_from_nodes(Φepw, X);
iAepw = RegularizedSVDPseudoInverse(Aepw; ϵ=1e-14);

The coefficients of the approximation are computed:

In [18]:
ξepw = solve_via_regularizedSVD(iAepw, b);
ũepw = (r,θ) -> sum([ξi * ϕi(r,θ) for (ξi, ϕi) in zip(ξepw, Φepw)]);

Absolute error function

In [19]:
eepw = (r,θ) -> ũepw(r,θ) - u(r,θ); eepw(1,π/2)

2.0077440043841932e-8 - 1.6604768334868574e-8im

Let's compute the relative residual:

In [20]:
resepw = norm(Aepw * ξepw - b) / norm(b)

6.538906705638461e-8

We get almost 8 digits of accuracy!
The size of the coefficients remains quite high:

In [21]:
nrmepw = norm(ξepw) / norm(U)

687669.95433013

### Down to machine precision

Let's double the number of EPW

In [22]:
N = 200

200

The above approximation process can be obtained much more rapidly with the
following convenience function

In [23]:
_, ξepw, ũepw, resepw, nrmepw, eepw = Dirichlet_sampling(k, U, N; smpl_type=sobol_sampling);

One can check that we obtain now a relative residual very close to machine
precision (13 digits of accuracy):

In [24]:
resepw

1.4303797155058453e-13

The size of the coefficients is also greatly reduced:

In [25]:
nrmepw

101.55244631916997

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*